In [1]:
from model_train_with_mlflow import NewsRecommendationModel

model1 = NewsRecommendationModel.load_from_checkpoint('latest_checkpoint.pth')
model1.eval()

c:\Users\alexm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NewsRecommendationModel(
  (news_encoder): NewsEncoder(
    (embedding): Embedding(31638, 100)
    (fc1): Linear(in_features=100, out_features=128, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=128, out_features=128, bias=True)
  )
  (user_encoder): UserEncoder(
    (attention): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=1, bias=True)
      (3): Softmax(dim=1)
    )
  )
  (criterion): BCELoss()
)

In [3]:
import torch

# Define dummy input for the model (adjust the size based on your model's input requirements)
dummy_input = {
    "news": torch.randint(0, 31638, (1, 10)),  # Example input for news_encoder
    "user": torch.randint(0, 31638, (1, 50))   # Example input for user_encoder
}

# Export the model to ONNX format
# Create dummy data based on the model's input requirements
batch_history = torch.randint(0, 31638, (1, 50))  # Example input for user history
batch_tokens = torch.randint(0, 31638, (1, 10))   # Example input for news tokens

# Export the model to ONNX format
torch.onnx.export(
    model1, 
    (batch_history, batch_tokens),  # Provide the inputs as a tuple
    "news_recommendation_model.onnx", 
    input_names=["batch_history", "batch_tokens"], 
    output_names=["output"], 
    dynamic_axes={
        "batch_history": {0: "batch_size", 1: "history_seq_len"},
        "batch_tokens": {0: "batch_size", 1: "tokens_seq_len"},
        "output": {0: "batch_size"}
    },
    opset_version=11
)

print("Model has been successfully converted to ONNX format.")

Model has been successfully converted to ONNX format.


In [7]:
from app import load_data
model, behaviors_data, news_data, news_features, tokenizer = load_data()

Loading behaviors and news data...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376471 entries, 0 to 376470
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   impression_id  376471 non-null  int64 
 1   user_id        376471 non-null  object
 2   time           376471 non-null  object
 3   history        365201 non-null  object
 4   impressions    376471 non-null  object
dtypes: int64(1), object(4)
memory usage: 14.4+ MB
None
Extracting news features...
Initializing tokenizer...
Vocabulary size: 31638
Data loading completed successfully.
Loading the model...
Model loaded successfully.


In [27]:
def test_model_with_onnx_weights(model, tokenizer, behaviors_data, news_data, news_features, user_id=None, history=None, num_recommendations=5, device='cpu'):
    """
    Test the model with .pth weights on a specific user or with custom history
    
    Args:
        model: Loaded NewsRecommendationModel
        tokenizer: Initialized tokenizer
        behaviors_data: Loaded behaviors data
        news_data: Loaded news data
        news_features: Extracted news features
        user_id: User ID from the dataset (if None, will use custom history)
        history: List of news IDs for custom history (required if user_id is None)
        num_recommendations: Number of recommendations to generate
        device: Device to run model on ('cpu' or 'cuda')
    
    Returns:
        List of recommended news items
    """
    # If user_id is provided, get their history from the dataset
    if user_id is not None:
        print(f"Looking up user {user_id}...")
        dev_behaviors = behaviors_data["dev"]
        test_behaviors = behaviors_data["test"]
        
        # Try to find user in dev or test behaviors
        print(dev_behaviors.info()) 
        user_data = dev_behaviors[dev_behaviors['user_id'] == user_id]
        if user_data.empty:
            user_data = test_behaviors[test_behaviors['user_id'] == user_id]
        
        if user_data.empty:
            print(f"User {user_id} not found in dataset")
            return None
        
        # Get user history
        user_history = user_data.iloc[0]['history'].split() if isinstance(user_data.iloc[0]['history'], str) and pd.notna(user_data.iloc[0]['history']) else []
        print(f"Found user with {len(user_history)} items in history")
    else:
        # Use provided history
        if history is None:
            print("Error: If user_id is not provided, history must be provided")
            return None
        user_history = history
        print(f"Using custom history with {len(history)} items")
    
    # Get candidate news IDs (all available news)
    candidate_news_ids = list(news_features.keys())
    
    # To speed up testing, limit the number of candidates
    max_candidates = 1000
    if len(candidate_news_ids) > max_candidates:
        print(f"Limiting candidates from {len(candidate_news_ids)} to {max_candidates} for faster processing")
        candidate_news_ids = candidate_news_ids[:max_candidates]
    else:
        print(f"Using {len(candidate_news_ids)} news items as candidates")
    
    print("Generating recommendations...")
    
    # SELF-CONTAINED RECOMMENDATION FUNCTION
    # Process user history
    max_history = 20
    history = user_history[:max_history]
    if len(history) < max_history:
        history += ['PAD'] * (max_history - len(history))
        
    # Process history titles
    history_tokens_list = []
    for h_news_id in history:
        title = news_features.get(h_news_id, {}).get('title', '') if h_news_id != 'PAD' else ''
        tokens = tokenizer.tokenize(title)
        history_tokens_list.append(tokens)
    
    history_tokens = torch.stack(history_tokens_list).unsqueeze(0).to(device)  # Add batch dimension
    
    # Process candidates and get scores
    candidate_scores = []
    
    # Process in batches for efficiency
    batch_size = 64
    for i in range(0, len(candidate_news_ids), batch_size):
        batch_news_ids = candidate_news_ids[i:i+batch_size]
        
        # Print progress
        if i % 200 == 0:
            print(f"Processing candidates {i} to {i+len(batch_news_ids)} of {len(candidate_news_ids)}")
        
        batch_tokens_list = []
        for news_id in batch_news_ids:
            title = news_features.get(news_id, {}).get('title', '')
            tokens = tokenizer.tokenize(title)
            batch_tokens_list.append(tokens)
        
        batch_tokens = torch.stack(batch_tokens_list).to(device)
        
        # We need to broadcast history_tokens to match batch_tokens
        batch_history = history_tokens.repeat(len(batch_news_ids), 1, 1).view(len(batch_news_ids), -1)
        
        with torch.no_grad():
            try:
                # Use ONNX runtime to run the model
                import onnxruntime as ort
                session = ort.InferenceSession(model.SerializeToString())
                
                # Prepare inputs for ONNX model
                inputs = {
                    "batch_history": batch_history.cpu().numpy(),
                    "batch_tokens": batch_tokens.cpu().numpy()
                }
                
                # Run inference
                outputs = session.run(None, inputs)
                scores = outputs[0]  # Assuming the first output is the scores
                
                for j, news_id in enumerate(batch_news_ids):
                    candidate_scores.append((news_id, scores[j]))
            except Exception as e:
                print(f"Error processing batch: {e}")
                continue
    
    # Sort by score
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    
    # Get top recommendations
    recommended_news = [news_id for news_id, _ in candidate_scores[:num_recommendations]]
    
    # Print recommendations
    print(f"\nTop {num_recommendations} recommendations:")
    actual_recommendations = [
        {"news_id": news_id, "title": news_features.get(news_id, {}).get('title', 'Unknown')}
        for news_id in recommended_news
    ]
    
    print(actual_recommendations)
    return actual_recommendations


In [29]:
from app import test_model_with_pth_weights
import pandas as pd

test_model_with_onnx_weights(onnx_model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')
test_model_with_onnx_weights(onnx_model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')

test_model_with_onnx_weights(onnx_model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')

test_model_with_onnx_weights(onnx_model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')
test_model_with_onnx_weights(onnx_model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')
test_model_with_onnx_weights(onnx_model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')

test_model_with_onnx_weights(onnx_model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')



Looking up user U254959...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376471 entries, 0 to 376470
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   impression_id  376471 non-null  int64 
 1   user_id        376471 non-null  object
 2   time           376471 non-null  object
 3   history        365201 non-null  object
 4   impressions    376471 non-null  object
dtypes: int64(1), object(4)
memory usage: 14.4+ MB
None
Found user with 57 items in history
Limiting candidates from 130379 to 1000 for faster processing
Generating recommendations...
Processing candidates 0 to 64 of 1000

Top 5 recommendations:
[{'news_id': 'N67526', 'title': 'Star Tracks: Celebs on Vacation'}, {'news_id': 'N86440', 'title': "Stowaway Discovered in Couple's Carry-On Luggage"}, {'news_id': 'N110516', 'title': "16 Live-Action Disney Movies in the Works After 'Maleficent: Mistress of Evil' (Photos)"}, {'news_id': 'N94988', 'title': 'L

[{'news_id': 'N67526', 'title': 'Star Tracks: Celebs on Vacation'},
 {'news_id': 'N86440',
  'title': "Stowaway Discovered in Couple's Carry-On Luggage"},
 {'news_id': 'N110516',
  'title': "16 Live-Action Disney Movies in the Works After 'Maleficent: Mistress of Evil' (Photos)"},
 {'news_id': 'N94988', 'title': 'Latest Automotive Safety Recalls'},
 {'news_id': 'N113706', 'title': 'Cooking advice you should never believe'}]

In [31]:
test_model_with_pth_weights(model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')
test_model_with_pth_weights(model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')
test_model_with_pth_weights(model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')
test_model_with_pth_weights(model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')
test_model_with_pth_weights(model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')
test_model_with_pth_weights(model, tokenizer, behaviors_data, news_data, news_features, user_id='U254959')

Looking up user U254959...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376471 entries, 0 to 376470
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   impression_id  376471 non-null  int64 
 1   user_id        376471 non-null  object
 2   time           376471 non-null  object
 3   history        365201 non-null  object
 4   impressions    376471 non-null  object
dtypes: int64(1), object(4)
memory usage: 14.4+ MB
None
Found user with 57 items in history
Limiting candidates from 130379 to 1000 for faster processing
Generating recommendations...
Processing candidates 0 to 64 of 1000

Top 5 recommendations:
[{'news_id': 'N67526', 'title': 'Star Tracks: Celebs on Vacation'}, {'news_id': 'N86440', 'title': "Stowaway Discovered in Couple's Carry-On Luggage"}, {'news_id': 'N110516', 'title': "16 Live-Action Disney Movies in the Works After 'Maleficent: Mistress of Evil' (Photos)"}, {'news_id': 'N94988', 'title': 'L

[{'news_id': 'N67526', 'title': 'Star Tracks: Celebs on Vacation'},
 {'news_id': 'N86440',
  'title': "Stowaway Discovered in Couple's Carry-On Luggage"},
 {'news_id': 'N110516',
  'title': "16 Live-Action Disney Movies in the Works After 'Maleficent: Mistress of Evil' (Photos)"},
 {'news_id': 'N94988', 'title': 'Latest Automotive Safety Recalls'},
 {'news_id': 'N113706', 'title': 'Cooking advice you should never believe'}]

In [29]:
model1.type

<bound method _DeviceDtypeModuleMixin.type of NewsRecommendationModel(
  (news_encoder): NewsEncoder(
    (embedding): Embedding(31638, 100)
    (fc1): Linear(in_features=100, out_features=128, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=128, out_features=128, bias=True)
  )
  (user_encoder): UserEncoder(
    (attention): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=1, bias=True)
      (3): Softmax(dim=1)
    )
  )
  (criterion): BCELoss()
)>

In [6]:
import onnx
import onnxruntime as ort

# Load the ONNX model
onnx_model = onnx.load("news_recommendation_model.onnx")
onnx.checker.check_model(onnx_model) 

In [5]:
!pip install onnxruntime

     --------------------------------------- 12.3/12.3 MB 32.8 MB/s eta 0:00:00
     ---------------------------------------- 46.0/46.0 kB ? eta 0:00:00
     ---------------------------------------- 86.8/86.8 kB ? eta 0:00:00
     ---------------------------------------- 83.2/83.2 kB 4.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
import pandas as pd
import onnxruntime as ort

def test_model_with_onnx_weights(
    model, tokenizer, behaviors_data, news_data, news_features,
    user_id=None, history=None, num_recommendations=5, device='cpu'
):
    """
    Test the model with ONNX weights using either a user ID or custom history.
    """
    # 1. Get user history
    if user_id:
        print(f"Looking up user {user_id}...")
        dev_behaviors = behaviors_data["dev"]
        test_behaviors = behaviors_data["test"]

        user_data = dev_behaviors[dev_behaviors['user_id'] == user_id]
        if user_data.empty:
            user_data = test_behaviors[test_behaviors['user_id'] == user_id]

        if user_data.empty:
            print(f"User {user_id} not found.")
            return None

        user_history = user_data.iloc[0]['history']
        user_history = user_history.split() if pd.notna(user_history) else []
    else:
        if not history:
            print("Error: Provide either a user_id or history.")
            return None
        user_history = history

    print(f"Using history with {len(user_history)} items")

    # 2. Trim/pad history
    max_history = 20
    history_ids = user_history[:max_history] + ['PAD'] * (max_history - len(user_history))

    history_titles = [
        news_features.get(nid, {}).get('title', '') if nid != 'PAD' else ''
        for nid in history_ids
    ]
    history_tokens = torch.stack([tokenizer.tokenize(title) for title in history_titles])
    history_tokens = history_tokens.unsqueeze(0).to(device)  # [1, 20, embedding_dim]

    # 3. Prepare candidate news
    candidate_news_ids = list(news_features.keys())
    max_candidates = 1000
    if len(candidate_news_ids) > max_candidates:
        print(f"Limiting candidates to {max_candidates}")
        candidate_news_ids = candidate_news_ids[:max_candidates]

    # 4. Initialize ONNX session
    try:
        session = ort.InferenceSession(model.SerializeToString())
    except Exception as e:
        print(f"ONNX Session error: {e}")
        return None

    candidate_scores = []
    batch_size = 64

    # 5. Batch process candidate news
    for i in range(0, len(candidate_news_ids), batch_size):
        batch_news_ids = candidate_news_ids[i:i+batch_size]
        print(f"Processing candidates {i}–{i+len(batch_news_ids)}")

        batch_titles = [news_features.get(nid, {}).get('title', '') for nid in batch_news_ids]
        batch_tokens = torch.stack([tokenizer.tokenize(title) for title in batch_titles]).to(device)

        # Repeat history for the batch
        batch_history = history_tokens.repeat(len(batch_news_ids), 1, 1).view(len(batch_news_ids), -1)

        # ONNX inference
        try:
            inputs = {
                "batch_history": batch_history.cpu().numpy(),
                "batch_tokens": batch_tokens.cpu().numpy()
            }
            outputs = session.run(None, inputs)
            scores = outputs[0]

            candidate_scores.extend(zip(batch_news_ids, scores))
        except Exception as e:
            print(f"Batch error: {e}")
            continue

    # 6. Select top recommendations
    top_candidates = sorted(candidate_scores, key=lambda x: x[1], reverse=True)[:num_recommendations]

    recommendations = [
        {"news_id": nid, "title": news_features.get(nid, {}).get('title', 'Unknown')}
        for nid, _ in top_candidates
    ]

    print(f"\nTop {num_recommendations} recommendations:")
    for rec in recommendations:
        print(rec)

    return recommendations
